# flowchart


<img src = 'crawler_flowchart.png'>

In [1]:
# import mysql.connector
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np
import sys
import re

In [2]:
sys.stdout = open('crawling_output.txt','w')

### 1. crawrler 

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")

time.sleep(2)


element = driver.find_element_by_name('q')

element.clear()
element.send_keys('google map')
element.submit()

element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

In [3]:
def get_info(html, shop_name, shop_address):
# shop 주소, 총 별점 갖고오기
    search_name, address, total_score, match=get_totalScore(html,shop_name, shop_address)
    
    # shop info가 있는가(addrsss로 확인)
    if address == '':
        review_list = ''
        score_list = ''
        return search_name, address,total_score, review_list, score_list, match 

    # 리뷰가 없는 경우    
    if  total_score == 0:
        review_list = 0
        score_list= 0
        
    # 리뷰가 있는 경우
    else:
        # 리뷰 text(버튼 )클릭
        review_button = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
        review_button.click()

        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤 다운
        try:
            element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
            element.click()

        except Exception as err:
            print(err)

        i = 0
        while  True:
            i += 1
            html = driver.find_element_by_tag_name('html')
            time.sleep(SCROLL_PAUSE_TIME)

            text1 = html.text
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.find_element_by_tag_name('html')
            text2 = html.text

            if text1 == text2:
                break
            else:
                continue
    
        review_html = driver.page_source

        review_list, score_list=get_review(review_html) 
        driver.back()

    return search_name, address, total_score, review_list, score_list, match

In [4]:
def get_review(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 개별 리뷰
    total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )

    review_list=[]
    for i, review in enumerate(total_review):
        review_list.append(review.text)

    #### 개별 별점
    scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )

    score_list = []
    for score in scores:
        score_list.append(score.attrs['aria-label'])

    return review_list, score_list

In [5]:
def start_crwarling(k, forpet_hash,shop_name, shop_address):
    print(k, shop_name, shop_address)

    # shop 이름 입력
    element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
    element.clear()
    
    search_word = shop_name + ' ' +shop_address[0]
#     search_word = shop_name
    element.send_keys(search_word)
    element.submit()
    
    search_button = driver.find_element_by_xpath('//button[@class="searchbox-searchbutton"]')
    search_button.click()
    
    time.sleep(SCROLL_PAUSE_TIME)

    # 키워드 입력 후, dict로 인덱스와 match 여부 받아오기
    select_shop_html = driver.page_source
    shop_select_dict= find_shop_by_name(select_shop_html, shop_name)
    
    # shoh이 여러개인 경우
    if len(shop_select_dict) != 0:
        print(f'shop이 {len(shop_select_dict)}개입니다')
        for index, name in shop_select_dict.items():
            shop_index =  2 * int(index) -1
            
            search_name = name
#             
            time.sleep(3)

            try:
                select_shop = driver.find_element_by_xpath(f'//div[@class = "section-layout section-scrollbox scrollable-y scrollable-show section-layout-flex-vertical"]/div/div[{shop_index}]/div[1]/div[1]')

                select_shop.click()

                time.sleep(SCROLL_PAUSE_TIME)

                html = driver.page_source

                search_name, address, total_score, review_list, score_list, match =get_info(html, shop_name, shop_address)
                print(f'{index}번째 shop:', search_name, address, total_score, match )
            except:
                search_name = shop_name
                address = ''
                total_score = ''
                match = 'not info'
                address = ''
                review_list = ''
                score_list = ''
           
            save_dataframe(forpet_hash, search_name, shop_address, address, total_score, review_list, score_list, match)
        
            time.sleep(SCROLL_PAUSE_TIME)
            driver.back()  
     
    # shop이 1개 인경우
    else:
        print('shop이 1개입니다')
        html = driver.page_source
        search_name, address, total_score, review_list, score_list, match =get_info(html, shop_name, shop_address)
        print(f'shop 1개, {search_name}, {address}, {match} ')
        
        save_dataframe(forpet_hash, search_name, shop_address, address, total_score, review_list, score_list, match)
    driver.back()  

    

In [6]:
# 여러개의  shop 목록에서 주소로 찾기
def find_shop_by_name(select_shop_html, shop_name):
    soup = BeautifulSoup(select_shop_html, 'html.parser')
#     print(select_shop_html)
    all_shop_name = soup.find_all('div', attrs = {'class': "section-result-title-container"} )
    
    shop_select_dict = {}
    for i, shop in enumerate(all_shop_name):
        include_word = ['동물', '펫', '애견', 'dot','cat','도그','캣','pet','야옹','멍멍']
        name_include_word = sum([ word in shop.text  for word in include_word]) 
        
        # 이름이 같은 경우
        if shop_name == shop.text :
            shop_select_dict[str(i+1)] = [shop.text, 'same_name']  
        # 이름이 비슷한 경우
        elif name_include_word >= 1:
            shop_select_dict[str(i+1)] = [shop.text,'simiral_name']
        else:
            pass
#     print(shop_select_dict)
    return shop_select_dict

In [7]:
# 주소와 total 별점 가져오기
def get_totalScore(html,shop_name, shop_address):
    
    
    soup = BeautifulSoup(html, 'html.parser')

    time.sleep(SCROLL_PAUSE_TIME)
    infos =soup.find_all('span',attrs = {'class': "section-info-text"} )
    time.sleep(SCROLL_PAUSE_TIME)
    
    # 검색 결과가 있는 경우
    try: 
        search_name = soup.find_all('div', attrs = {'class':'section-hero-header-title-description' })[0].text.replace(' ','')
        if shop_name in search_name:
            search_name = shop_name
        else:
            p = re.compile("[가-힣]")
            search_name = ''.join(p.findall(search_name)).replace('주','')
        
        print('serach_name', search_name, 'shop_name', shop_name)
        
        #  shop이름이 일치하는 경우
        if shop_name in search_name:
            # shop 정보가 없는 경우
            if len(infos)== 0:
                total_score = ''
                match = 'not info'
                address = ''

                return address, total_score
            # shop 정보가 있는 경우    
            else:
                address =infos[0].text

                gu = shop_address[0] in address
                dong = shop_address[1] in address
                bungi = shop_address[2] in address


                if (gu and dong and bungi) or (dong and bungi):
                    match = 'match_all'
                elif (gu and dong) or dong:
                    match = 'match_dong'
                elif gu:
                    match = 'match_gu'
                else:
                    match = 'not_match' 
                    
                    
                    
        #shop이름이 다른 경우
        else:
            address =infos[0].text
            match = 'x'


        total_score = soup.find_all('div',attrs = {'class': "gm2-display-2"} )

        if len(total_score)!=0:
            total_score = total_score[0].text
        else:
            total_score = 0
    
    # 검색 결과가 없는 경우
    except:
        total_score = ''
        match = 'not info'
        address = ''
        search_name = shop_name
    
    return search_name, address, total_score, match

In [8]:
def save_dataframe(forpet_hash, search_name, shop_address, address, total_score, review_list, score_list, match):
    
    # shop 정보가 없는 경우
    if address == '':
        not_info_table_insert(not_info_table, forpet_hash, search_name, match)
    
    # shop 정보가 있는 경우
    else:
        # match 여부에 따라
        if match == 'match_all':
            table_total = match_total_table
            table_review = match_review_table

        elif match == 'x':
            table_total = x_total_table
            table_review = x_review_table

        # shop 이름만 같은 경우, match = match_dong, match_gu, not_match
        else:    
            table_total = not_match_total_table
            table_review = not_match_review_table

        total_table_insert(table_total, forpet_hash, search_name, shop_address, address, total_score, review_list, match)
        review_table_insert(table_review, forpet_hash, search_name, shop_address, address, review_list, score_list, match)   


### 2. table
#### 1. total table

In [12]:
not_match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'address','total_score', 'review_num','match'])
not_match_total_table

,forpet_hash,shop_name,shop_address,address,total_score,review_num,match


In [13]:
match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'address','total_score', 'review_num','match'])
match_total_table

,forpet_hash,shop_name,shop_address,address,total_score,review_num,match


In [14]:
x_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'address','total_score', 'review_num','match'])
x_total_table

,forpet_hash,shop_name,shop_address,address,total_score,review_num,match


In [15]:
def total_table_insert(total_table, forpet_hash, search_name, shop_address, address, total_score, review_list, match):
    if len(total_table[total_table['address']== address]) == 0:
        # shop 정보가 없고, total_sclre가 없는 경우
        if total_score == '':
            total_table.loc[len(total_table)+1] = [forpet_hash, search_name, shop_address,  address, np.nan, np.nan, match]
        # shop 정보가 있고, total_score가 있는 경우
        elif total_score == 0:
            total_table.loc[len(total_table)+1] = [forpet_hash, search_name, shop_address, address, 0, 0, match]

        #total score이 없는 경우
        else:
            total_table.loc[len(total_table)+1] = [forpet_hash, search_name, shop_address, address,total_score, len(review_list), match]
        print(f'total_table: {search_name}, {address} 등록완료')
    else:
        print(f'total_table: {search_name}, {address} 이미 있음')
    return total_table

#### 2. review table

In [16]:
not_match_review_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name','shop_address','address', 'num', 'review', 'score', 'match'])
not_match_review_table

,forpet_hash,shop_name,shop_address,address,num,review,score,match


In [17]:
match_review_table= pd.DataFrame(columns = ['forpet_hash','shop_name','shop_address','address', 'num', 'review', 'score', 'match'])
match_review_table

,forpet_hash,shop_name,shop_address,address,num,review,score,match


In [18]:
x_review_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name','shop_address','address', 'num', 'review', 'score', 'match'])
x_review_table

,forpet_hash,shop_name,shop_address,address,num,review,score,match


In [19]:
def review_table_insert(review_table, forpet_hash, search_name, shop_address, address, review_list, score_list, match):
    if len(review_table[review_table['address']== address]) == 0:
        # shop 정보가 없고, total_sclre가 없는 경우
        if review_list == '':
            review_table.loc[len(review_table)+1] = [forpet_hash, search_name, shop_address, address, 1,  np.nan, np.nan, match]
            print(f'review_table: {search_name}, {address} 등록완료')

        # shop 정보가 있고, total_score가 있는 경우
        elif review_list == 0:
            review_table.loc[len(review_table)+1] = [forpet_hash, search_name, shop_address, address, 1, 0, 0, match]
            print(f'review_table: {search_name}, {address} 등록완료')

        #total score이 없는 경우
        else:
            for i, (review, score, match) in enumerate(zip(review_list, score_list, match)):
                review_table.loc[i+1+len(review_table)] = [forpet_hash, search_name, shop_address, address, i, review, score, match]
            print(f'review_table: {search_name}, {address}, {len(review_list)} 등록완료')
    else:
        print(f'review_table: {search_name}, {address} 이미 있음')

    return review_table

#### 3.not_info_table

In [20]:
not_info_table = pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'match'])
not_info_table

,forpet_hash,shop_name,match


In [21]:
def not_info_table_insert(not_info_table, forpet_hash, shop_name, match):
    not_info_table.loc[1+len(not_info_table)] = [forpet_hash, shop_name, match]
    print(f'not_info_table: {shop_name} 등록완료')
    return not_info_table

### 3. ordinary table

In [22]:
ordinary_table = pd.read_csv('data/mysql_forpet_shop.csv', index_col = 0)
ordinary_table.head()

,forpet_hash,address_name,place_name
0,4b793eb834462b029e435eaeb49433efc61755c9e15a24...,서울 종로구 평창동 530-8,쿠나이앤티
1,f5169c15a126e4059f0a89bb80fccd236693d5056d24ae...,서울 종로구 창신동 468-1,애견용품할인매장
2,aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb...,서울 종로구 창신동 436-64,펫클럽 청계점
3,2806f173e3a783ed14d74c9a83da2f62c34f45d470e893...,서울 종로구 홍지동 103-24,나니스펫푸드
4,aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c...,서울 종로구 필운동 137-5,서촌양품


In [10]:
len(ordinary_table)

8582

In [24]:
SCROLL_PAUSE_TIME = 2
all_cnt = 0
for k in range(6535, len(ordinary_table)):
    forpet_hash, address_name, place_name= ordinary_table.loc[k] 
  
    shop_address =address_name.split(' ')[1:]
    shop_name = place_name.replace(' ','')
    start_crwarling(k, forpet_hash, shop_name, shop_address)
    
    print('\n')
    try:
        back_button = driver.find_element_by_xpath('//button[@class="ozj7Vb3wnYq__action-button-clickable"]')
        back_button.click()
    except:
        pass
    
#     if k % 100 == 0:
#         match_total_table.to_csv('match_total_table.csv')
#         not_match_total_table.to_csv('not_match_total_table.csv')
#         match_review_table.to_csv('match_review_table.csv')
#         not_match_review_table.to_csv('not_match_review_table.csv')
#         x_total_table.to_csv('x_total_table.csv')
#         x_review_table.to_csv('x_review_table.csv')
#         not_info_table.to_csv('not_info_table.csv')        

6535 초이동물병원 ['성남시', '수정구', '신흥1동', '5168']


InvalidElementStateException: Message: invalid element state: Element is not currently interactable and may not be manipulated
  (Session info: chrome=80.0.3987.163)


In [54]:



print('match_total_table\n',match_total_table)
print('not_match_total_table\n',not_match_total_table)
print('match_review_table\n',match_review_table)
print('not_match_review_table\n',not_match_review_table)
print('x_total_table\n', x_total_table)
print('x_review_table\n', x_review_table)


match_total_table
                                          forpet_hash shop_name  \
1  aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb...   펫클럽 청계점   
2  2806f173e3a783ed14d74c9a83da2f62c34f45d470e893...    나니스펫푸드   
3  aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c...      서촌양품   
4  fb83c79c4e90759c7c48970c543e2f8671e1e39d3eb76d...      뚜리까까   

                    shop_address total_score  match  
1  서울특별시 종로구 창신동 436-64번지 일신파워빌딩         NaN  match  
2   홍지동 103-24번지 1층 종로구 서울특별시 KR         NaN  match  
3            서울특별시 종로구 필운동 137-5         NaN  match  
4     서울특별시 성북구 동소문동3가 동소문로2길 79         NaN  match  
not_match_total_table
                                          forpet_hash    shop_name  \
1  f5169c15a126e4059f0a89bb80fccd236693d5056d24ae...     애견용품할인매장   
2  8aa63cf25e3842284a45ab5d4f603e816dba32c9406b88...      푸드마스터그룹   
3  bd8ce4fe5ec9031f0d986ac8e3091594a104e02d9e1d71...         도그스푼   
4  ad85ca27faea5e1533bdd8873a5a366d237ffd1311f510...    스프링펫 서대문점   
5  57eed23d4